# ChatGitHubCopilot

This will help you getting started with GitHubCopilot [chat models](/docs/concepts/#chat-models). For detailed documentation of all ChatGitHubCopilot features and configurations head to the [API reference](https://api.python.langchain.com/en/latest/chat_models/langchain_github_copilot.chat_models.ChatGitHubCopilot.html).

## Overview
### Integration details

| Class | Package | Local | Serializable | JS support| Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [ChatGitHubCopilot](https://api.python.langchain.com/en/latest/chat_models/langchain_github_copilot.chat_models.ChatGitHubCopilot.html) | [langchain-github-copilot](https://api.python.langchain.com/en/latest/github_copilot_api_reference.html) | ❌ | ❌ | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-github-copilot?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-github-copilot?style=flat-square&label=%20) |

### Model features
| [Tool calling](/docs/how_to/tool_calling) | [Structured output](/docs/how_to/structured_output/) | JSON mode | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ✅ | ❌ | 

## Setup

To access GitHubCopilot models you'll need to create a GitHub account, get a GitHub token, and install the `langchain-github-copilot` integration package.

### Credentials

Use the following function to set up your GitHub Token as an environment variable.

In [ ]:
import requests
import time

GITHUB_CLIENT_ID = "Iv1.b507a08c87ecfe98"
HEADERS = {
    'accept': 'application/json',
    'editor-version': 'Neovim/0.6.1',
    'editor-plugin-version': 'copilot.vim/1.16.0',
    'content-type': 'application/json',
    'user-agent': 'GitHubCopilot/1.155.0',
    'accept-encoding': 'gzip,deflate,br'
}

def setup():
    access_token = get_access_token()
    token = get_token(access_token)
    
    with open('.env', 'a') as f:
        f.write(f'\nGITHUB_TOKEN="{token}"\n')

    print('Authentication success!')

def get_token(access_token):
    response = requests.get(
        'https://api.github.com/copilot_internal/v2/token',
        headers={**HEADERS, 'authorization': f'token {access_token}'}
    )
    response.raise_for_status()
    return response.json().get('token')

def get_access_token():
    response = requests.post(
        'https://github.com/login/device/code',
        headers=HEADERS,
        json={'client_id': GITHUB_CLIENT_ID, 'scope': 'read:user'}
    )
    response.raise_for_status()
    data = response.json()
    device_code = data.get('device_code')
    user_code = data.get('user_code')
    verification_uri = data.get('verification_uri')

    print(f'Please visit {verification_uri} and enter code {user_code} to authenticate.')

    while True:
        time.sleep(5)
        response = requests.post(
            'https://github.com/login/oauth/access_token',
            headers=HEADERS,
            json={
                'client_id': GITHUB_CLIENT_ID,
                'device_code': device_code,
                'grant_type': 'urn:ietf:params:oauth:grant-type:device_code'
            }
        )
        response.raise_for_status()
        access_token = response.json().get('access_token')
        if access_token:
            return access_token

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")

### Installation

The LangChain GitHubCopilot integration lives in the `langchain-github-copilot` package:

In [ ]:
%pip install -qU langchain-github-copilot

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_github_copilot import ChatGitHubCopilot

llm = ChatGitHubCopilot(
    temperature=0.9,
    max_tokens=100,
    timeout= None,
    stop= ["<END>"]
)

## Invocation

In [4]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 5, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 31, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 36}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop'}, id='run-a3651d31-79e6-4318-bc76-109af65c4213-0', usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
print(ai_msg.content)

J'adore la programmation.


## Chaining

We can [chain](/docs/how_to/sequence/) our model with a prompt template like so:

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

AIMessage(content='Ich liebe Programmieren.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 5, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens': 26, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}, 'total_tokens': 31}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop'}, id='run-04da3104-cac8-44c7-8c9b-a89d4c1f9f51-0', usage_metadata={'input_tokens': 26, 'output_tokens': 5, 'total_tokens': 31, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## API reference

For detailed documentation of all ChatGitHubCopilot features and configurations head to the API reference: https://api.python.langchain.com/en/latest/chat_models/langchain_github_copilot.chat_models.ChatGitHubCopilot.html